In [33]:
import csv
import math

In [34]:
def prob(e, S):
    return S[e]/sum(S.values())

def info(e, S):
    return -math.log2(prob(e, S))

def entropia(S):
    return sum([info(e, S) * prob(e, S) for e in S])

In [35]:
S1 = {}
direcciones = {'BROADCAST': 0, 'UNICAST': 0}
protocolos = {}
total = 20000

with open('output.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, )
    next(reader, None)  # skip the headers
    for [dire, proto, count] in reader:
        S1[(dire, int(proto))] = int(count)
        direcciones[dire] += int(count)
        if proto not in protocolos:
            protocolos[proto] = 0
        protocolos[proto] += int(count)


• Porcentaje de tráfico Broadcast/Unicast sobre el tráfico total.


In [36]:
ratio_broadcast = direcciones['BROADCAST'] / total
print(f'porcentaje broadcast: {ratio_broadcast*100}%')
print(f'porcentaje unicast: {100 - ratio_broadcast*100}%')

porcentaje broadcast: 0.0%
porcentaje unicast: 100.0%


• Porcentaje de aparición de cada protocolo encontrado.


In [37]:
for (proto, count) in protocolos.items():
  print(f'porcentaje protocolo {proto}: {(count / total)*100}%')

porcentaje protocolo 2048: 99.72%
porcentaje protocolo 2054: 0.27999999999999997%


• Entropía de cada red analizada.


In [38]:
 
print(f'entropia: {entropia(S1)}')

entropia: 0.027778886345163643


• Cantidad de información de cada símbolo comparado con la entropía de la red.


In [40]:
for ((dire, proto), count) in S1.items():
  print(f'info de evento ({dire},{proto}): {info((dire, proto), S1)}')

#a que se referirá con "comparado con"?? desconozco

info de evento (UNICAST,2048): 0.004045212057948735
info de evento (UNICAST,2054): 8.480357457491845
